In [ ]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
# !pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
# !pip install cython pyyaml==5.1
# !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print ('torch',torch.__version__)
print ('torchvision',torchvision.__version__)
!gcc --version
# opencv is pre-installed on colab

In [ ]:
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

# Mengmeng: cu100 means cuda v10.0? If have problem please install from source:
# https://github.com/facebookresearch/detectron2/blob/master/INSTALL.md

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

import matplotlib.pyplot as plt

In [ ]:
# setup model
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

# Detect a picture

In [ ]:
# !wget http://images.cocodataset.org/val2017/000000439715.jpg -O data/test.jpg
im = cv2.imread("/home/ubuntu/git-clones/AutoCheckout-CMU-F4/data/cps-test-videos/192.168.1.100_2020-01-28_22-24-06/frame0.jpg")
# mengmeng: wrong color https://www.pyimagesearch.com/2014/11/03/display-matplotlib-rgb-image/
# plt.imshow(im)
# plt.show()

im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
plt.imshow(im)
plt.show()

Then, we create a detectron2 config and a detectron2 DefaultPredictor to run inference on this image.

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
outputs = predictor(im)
# outputs["instances"].pred_classes
# outputs["instances"].pred_boxes

# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (20,10))
plt.imshow(v.get_image()[:, :, ::-1])

# Detect a video live

In [ ]:
import PIL.Image
import io
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import cv2
import time

def showarray(a, fmt='jpeg'):
    a = np.uint8(a)
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
# cam = cv2.VideoCapture(0)
cam = cv2.VideoCapture('/home/ubuntu/git-clones/AutoCheckout-CMU-F4/data/cps-test-videos/video-clip.mp4')
# cam = cv2.VideoCapture('/home/ubuntu/git-clones/AutoCheckout-CMU-F4/data/cps-test-videos/192.168.1.100_2020-01-28_22-24-06.mp4')
frame_count = 0
start_time = time.time()
success = True
# out = cv2.VideoWriter('data/cps-test-videos/video-clip-output.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 10, (1280,720))
try:
    while(success):
        # Capture frame-by-frame
        success, frame = cam.read()
# Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        outputs = predictor(frame)
        # We can use `Visualizer` to draw the predictions on the image.
        v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#         plt.figure(figsize = (20,10))
#         plt.imshow(v.get_image()[:, :, ::-1])
        
        showarray(v.get_image()[:, :, ::-1])
#         out.write(v.get_image()[:, :, ::-1])
        frame_count += 1
        elapsed_time = time.time() - start_time # seconds
        print (int(frame_count/elapsed_time), 'fps')
# Display the frame until new frame is available
        clear_output(wait=True)
        
        
except KeyboardInterrupt:
    cam.release()
#     out.release()
    print ("Stream stopped")

# Detect a video offline

In [ ]:
# !git clone https://github.com/facebookresearch/detectron2
# !cd detectron2 
!cd ../detectron2 && python3 demo/demo.py --config-file configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml --video-input /home/ubuntu/git-clones/AutoCheckout-CMU-F4/data/cps-test-videos/video-clip.mp4 --confidence-threshold 0.6 --output /home/ubuntu/git-clones/AutoCheckout-CMU-F4/data/cps-test-videos/video-clip-output.mkv \
  --opts MODEL.WEIGHTS detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_101_3x/139514519/model_final_cafdb1.pkl

In [ ]:
# display a video
from IPython.display import Video

Video('data/cps-test-videos/video-clip.mp4')